In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install bertviz

from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AdamW

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

import pickle
import numpy as np

from tqdm import trange

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 7.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manyl

# **Open the data**

The data has been already split into train and test:

In [ ]:
#f_in = open("drive/MyDrive/data/7-classes/train_set_paragraphs.pkl","rb")
f_in = open("drive/MyDrive/data/7-classes/train_set_paragraphs_augm-19.pkl","rb")
#f_in = open("drive/MyDrive/data/7-classes/train_set_sentences.pkl","rb")
#f_in = open("drive/MyDrive/data/7-classes/train_set_sentences_augm-8.pkl","rb")

train_data = pickle.load(f_in)

f_in.close()

f_in = open("drive/MyDrive/data/7-classes/test_set_paragraphs.pkl","rb")
#f_in = open("drive/MyDrive/data/7-classes/test_set_sentences.pkl","rb")

test_data = pickle.load(f_in)

f_in.close()

# **Training**

In [ ]:
average_results = []

# Fix random seed to make results reproducible:
torch.manual_seed(256)

# Repeat experiment 5 times, report averaged results:
for j in range(5):
    print("\nExperiment",j+1,"--------------------------------------------------------------")
    # Data preprocessing:

    model_name = "bert-base-uncased"

    # Define tokenizer:
    tokenizer = BertTokenizer.from_pretrained(model_name,do_lower_case=True)

    # Tokenize training and test sets:
    tokenizer_train = tokenizer([train_data[i][1] for i in range(len(train_data))], padding="longest", truncation = True, return_tensors="pt")

    tokenizer_test = tokenizer([test_data[i][1] for i in range(len(test_data))], padding="longest", truncation = True, return_tensors="pt")

    # Define Dataloaders:
    batch_size = 16

    train_set = TensorDataset(tokenizer_train['input_ids'],
                              tokenizer_train['attention_mask'],
                              torch.tensor([int(train_data[i][2]) for i in range(len(train_data))]))

    test_set = TensorDataset(tokenizer_test['input_ids'],
                             tokenizer_test['attention_mask'],
                             torch.tensor([int(test_data[i][2]) for i in range(len(test_data))]))

    train_dataloader = DataLoader(
                train_set,
                sampler = RandomSampler(train_set),
                batch_size = batch_size
            )

    validation_dataloader = DataLoader(
                test_set,
                sampler = SequentialSampler(test_set),
                batch_size = batch_size
            )

    # Define the model:
    device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

    model = BertForSequenceClassification.from_pretrained(model_name, num_labels=7)

    model.to(device)

    # Define parameters and metrics to optimize:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]

    optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5, eps=10e-8)

    def flatten_preds(preds):
      preds = np.argmax(preds, axis = 1).flatten()

      return preds.tolist()

    # Training and validation:
    epochs = 10

    train_loss_set = []
    count = 0
    best_acc_weight = 0
    best_results = []
    # trange is a tqdm wrapper around the normal python range
    for _ in trange(epochs, desc="Epoch"):
        # Tracking variables for training
        tr_loss = 0
        nb_tr_examples, nb_tr_steps = 0, 0

        # Train the model
        model.train()
        for step, batch in enumerate(train_dataloader):
            # Add batch to device CPU or GPU
            batch = tuple(t.to(device) for t in batch)
            # Unpack the inputs from our dataloader
            b_input_ids, b_input_mask, b_labels = batch
            # Clear out the gradients (by default they accumulate)
            optimizer.zero_grad()
            # Forward pass
            outputs = model(b_input_ids,token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
            # Get loss value
            loss = outputs[0]
            # Add it to train loss list
            train_loss_set.append(loss.item())
            # Backward pass
            loss.backward()
            # Update parameters and take a step using the computed gradient
            optimizer.step()

            # Update tracking variables
            tr_loss += loss.item()
            nb_tr_examples += b_input_ids.size(0)
            nb_tr_steps += 1

        print("\nTrain loss: {}".format(tr_loss/nb_tr_steps))

        # Tracking variables for validation
        val_preds = []
        val_labels = []
        # Validation of the model
        model.eval()
        # Evaluate data for one epoch
        for batch in validation_dataloader:
            # Add batch to device CPU or GPU
            batch = tuple(t.to(device) for t in batch)
            # Unpack the inputs from our dataloader
            b_input_ids, b_input_mask, b_labels = batch
            # Telling the model not to compute or store gradients, saving memory and speeding up validation
            with torch.no_grad():
                # Forward pass, calculate logit predictions
                outputs =  model(b_input_ids,token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
                loss, logits = outputs[:2]
            # Move logits and labels to CPU
            logits = logits.detach().cpu().numpy()
            labels = b_labels.to('cpu').numpy()
            # Update predicted and true labels
            val_preds += flatten_preds(logits)
            val_labels += labels.tolist()

        # Cost-based validation:
        validation_cost = np.zeros(len(val_labels),float)
        validation_cost+=1
        weight_1 = len(val_labels)/float(7)/float(len([i for i in val_labels if i==1]))
        weight_2 = len(val_labels)/float(7)/float(len([i for i in val_labels if i==2]))
        weight_3 = len(val_labels)/float(7)/float(len([i for i in val_labels if i==3]))
        weight_4 = len(val_labels)/float(7)/float(len([i for i in val_labels if i==4]))
        weight_5 = len(val_labels)/float(7)/float(len([i for i in val_labels if i==5]))
        weight_6 = len(val_labels)/float(7)/float(len([i for i in val_labels if i==6]))
        for i in range(len(val_labels)):
            if val_labels[i] == 1:
                validation_cost[i] = weight_1
            elif val_labels[i] == 2:
                validation_cost[i] = weight_2
            elif val_labels[i] == 3:
                validation_cost[i] = weight_3
            elif val_labels[i] == 4:
                validation_cost[i] = weight_4
            elif val_labels[i] == 5:
                validation_cost[i] = weight_5
            elif val_labels[i] == 6:
                validation_cost[i] = weight_6

        # Calculate validation metrics:
        val_acc = accuracy_score(val_labels,val_preds)
        val_acc_weight = accuracy_score(val_labels,val_preds,sample_weight=validation_cost)
        val_prec = precision_score(val_labels,val_preds,average=None)
        val_rec = recall_score(val_labels,val_preds,average=None)
        val_f1 = f1_score(val_labels,val_preds,average=None)
        count+=1

        # Save best results:
        if val_acc_weight > best_acc_weight:
            best_acc_weight = val_acc_weight
            best_results = (count, val_acc, val_acc_weight, val_prec, val_rec, val_f1)

        # Calculate validation metrics:
        print("\nEpoch: ",count)
        print("\t - Validation Accuracy: {:.4f}".format(val_acc))
        print("\t - Validation Accuracy (weighted): {:.4f}\n".format(val_acc_weight))
        print("\t - Validation Precision (class 0): {:.4f}".format(val_prec[0]))
        print("\t - Validation Recall (class 0): {:.4f}".format(val_rec[0]))
        print("\t - Validation F1-score (class 0): {:.4f}\n".format(val_f1[0]))
        print("\t - Validation Precision (class 1): {:.4f}".format(val_prec[1]))
        print("\t - Validation Recall (class 1): {:.4f}".format(val_rec[1]))
        print("\t - Validation F1-score (class 1): {:.4f}\n".format(val_f1[1]))
        print("\t - Validation Precision (class 2): {:.4f}".format(val_prec[2]))
        print("\t - Validation Recall (class 2): {:.4f}".format(val_rec[2]))
        print("\t - Validation F1-score (class 2): {:.4f}\n".format(val_f1[2]))
        print("\t - Validation Precision (class 3): {:.4f}".format(val_prec[3]))
        print("\t - Validation Recall (class 3): {:.4f}".format(val_rec[3]))
        print("\t - Validation F1-score (class 3): {:.4f}\n".format(val_f1[3]))
        print("\t - Validation Precision (class 4): {:.4f}".format(val_prec[4]))
        print("\t - Validation Recall (class 4): {:.4f}".format(val_rec[4]))
        print("\t - Validation F1-score (class 4): {:.4f}\n".format(val_f1[4]))
        print("\t - Validation Precision (class 5): {:.4f}".format(val_prec[5]))
        print("\t - Validation Recall (class 5): {:.4f}".format(val_rec[5]))
        print("\t - Validation F1-score (class 5): {:.4f}\n".format(val_f1[5]))
        print("\t - Validation Precision (class 6): {:.4f}".format(val_prec[6]))
        print("\t - Validation Recall (class 6): {:.4f}".format(val_rec[6]))
        print("\t - Validation F1-score (class 6): {:.4f}\n".format(val_f1[6]))

    # Best results of an epoch:
    print("\nBest results:")
    print("\nEpoch: ",best_results[0])
    print("\t - Validation Accuracy: {:.4f}".format(best_results[1]))
    print("\t - Validation Accuracy (weighted): {:.4f}\n".format(best_results[2]))
    print("\t - Validation Precision (class 0): {:.4f}".format(best_results[3][0]))
    print("\t - Validation Recall (class 0): {:.4f}".format(best_results[4][0]))
    print("\t - Validation F1-score (class 0): {:.4f}\n".format(best_results[5][0]))
    print("\t - Validation Precision (class 1): {:.4f}".format(best_results[3][1]))
    print("\t - Validation Recall (class 1): {:.4f}".format(best_results[4][1]))
    print("\t - Validation F1-score (class 1): {:.4f}\n".format(best_results[5][1]))
    print("\t - Validation Precision (class 2): {:.4f}".format(best_results[3][2]))
    print("\t - Validation Recall (class 2): {:.4f}".format(best_results[4][2]))
    print("\t - Validation F1-score (class 2): {:.4f}\n".format(best_results[5][2]))
    print("\t - Validation Precision (class 3): {:.4f}".format(best_results[3][3]))
    print("\t - Validation Recall (class 3): {:.4f}".format(best_results[4][3]))
    print("\t - Validation F1-score (class 3): {:.4f}\n".format(best_results[5][3]))
    print("\t - Validation Precision (class 4): {:.4f}".format(best_results[3][4]))
    print("\t - Validation Recall (class 4): {:.4f}".format(best_results[4][4]))
    print("\t - Validation F1-score (class 4): {:.4f}\n".format(best_results[5][4]))
    print("\t - Validation Precision (class 5): {:.4f}".format(best_results[3][5]))
    print("\t - Validation Recall (class 5): {:.4f}".format(best_results[4][5]))
    print("\t - Validation F1-score (class 5): {:.4f}\n".format(best_results[5][5]))
    print("\t - Validation Precision (class 6): {:.4f}".format(best_results[3][6]))
    print("\t - Validation Recall (class 6): {:.4f}".format(best_results[4][6]))
    print("\t - Validation F1-score (class 6): {:.4f}\n".format(best_results[5][6]))
    average_results.append(best_results)

# Determine best weighted accuracy:
max_acc_weight = np.max([i[2] for i in average_results])
index_best = [i for i in range(len(average_results)) if average_results[i][2]==max_acc_weight][0]

# Output best and averaged results:
print("\n-------------------------------------------------------")
print("\t - Averaged F1-score (class 0): {:.4f}".format(np.mean([i[5][0] for i in average_results])))
print("\t - Best F1-score (class 0): {:.4f}\n".format(average_results[index_best][5][0]))
print("\t - Averaged F1-score (class 1): {:.4f}".format(np.mean([i[5][1] for i in average_results])))
print("\t - Best F1-score (class 1): {:.4f}\n".format(average_results[index_best][5][1]))
print("\t - Averaged F1-score (class 2): {:.4f}".format(np.mean([i[5][2] for i in average_results])))
print("\t - Best F1-score (class 2): {:.4f}\n".format(average_results[index_best][5][2]))
print("\t - Averaged F1-score (class 3): {:.4f}".format(np.mean([i[5][3] for i in average_results])))
print("\t - Best F1-score (class 3): {:.4f}\n".format(average_results[index_best][5][3]))
print("\t - Averaged F1-score (class 4): {:.4f}".format(np.mean([i[5][4] for i in average_results])))
print("\t - Best F1-score (class 4): {:.4f}\n".format(average_results[index_best][5][4]))
print("\t - Averaged F1-score (class 5): {:.4f}".format(np.mean([i[5][5] for i in average_results])))
print("\t - Best F1-score (class 5): {:.4f}\n".format(average_results[index_best][5][5]))
print("\t - Averaged F1-score (class 6): {:.4f}".format(np.mean([i[5][6] for i in average_results])))
print("\t - Best F1-score (class 6): {:.4f}\n".format(average_results[index_best][5][6]))
print("\t - Max weighted accuracy: {:.4f}\n".format(max_acc_weight))

print("Line to copy to .xls:")
print((str(average_results[index_best][3][0])+'\t'+str(average_results[index_best][4][0])+'\t'+str(average_results[index_best][5][0])+'\t'+str(average_results[index_best][3][1])+'\t'+str(average_results[index_best][4][1])+'\t'+str(average_results[index_best][5][1])+'\t'+str(average_results[index_best][3][2])+'\t'+str(average_results[index_best][4][2])+'\t'+str(average_results[index_best][5][2])+'\t'+str(average_results[index_best][3][3])+'\t'+str(average_results[index_best][4][3])+'\t'+str(average_results[index_best][5][3])+'\t'+str(average_results[index_best][3][4])+'\t'+str(average_results[index_best][4][4])+'\t'+str(average_results[index_best][5][4])+'\t'+str(average_results[index_best][3][5])+'\t'+str(average_results[index_best][4][5])+'\t'+str(average_results[index_best][5][5])+'\t'+str(average_results[index_best][3][6])+'\t'+str(average_results[index_best][4][6])+'\t'+str(average_results[index_best][5][6])+'\t'+str(max_acc_weight)).replace('.',','))


Experiment 1 --------------------------------------------------------------


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...


Epoch:  8
	 - Validation Accuracy: 0.8505
	 - Validation Accuracy (weighted): 0.8466

	 - Validation Precision (class 0): 0.8667
	 - Validation Recall (class 0): 0.5200
	 - Validation F1-score (class 0): 0.6500

	 - Validation Precision (class 1): 0.8636
	 - Validation Recall (class 1): 0.9048
	 - Validation F1-score (class 1): 0.8837

	 - Validation Precision (class 2): 0.7407
	 - Validation Recall (class 2): 1.0000
	 - Validation F1-score (class 2): 0.8511

	 - Validation Precision (class 3): 0.8636
	 - Validation Recall (class 3): 1.0000
	 - Validation F1-score (class 3): 0.9268

	 - Validation Precision (class 4): 1.0000
	 - Validation Recall (class 4): 0.9091
	 - Validation F1-score (class 4): 0.9524

	 - Validation Precision (class 5): 0.8571
	 - Validation Recall (class 5): 1.0000
	 - Validation F1-score (class 5): 0.9231

	 - Validation Precision (class 6): 1.0000
	 - Validation Recall (class 6): 0.8000
	 - Validation F1-score (class 6): 0.8889


Train loss: 0.0338647014207460

Epoch:  90%|█████████ | 9/10 [10:09<01:07, 67.67s/it]


Epoch:  9
	 - Validation Accuracy: 0.7664
	 - Validation Accuracy (weighted): 0.7453

	 - Validation Precision (class 0): 0.5600
	 - Validation Recall (class 0): 0.5600
	 - Validation F1-score (class 0): 0.5600

	 - Validation Precision (class 1): 0.8333
	 - Validation Recall (class 1): 0.7143
	 - Validation F1-score (class 1): 0.7692

	 - Validation Precision (class 2): 0.7500
	 - Validation Recall (class 2): 0.9000
	 - Validation F1-score (class 2): 0.8182

	 - Validation Precision (class 3): 0.8636
	 - Validation Recall (class 3): 1.0000
	 - Validation F1-score (class 3): 0.9268

	 - Validation Precision (class 4): 0.8889
	 - Validation Recall (class 4): 0.7273
	 - Validation F1-score (class 4): 0.8000

	 - Validation Precision (class 5): 0.8333
	 - Validation Recall (class 5): 0.8333
	 - Validation F1-score (class 5): 0.8333

	 - Validation Precision (class 6): 1.0000
	 - Validation Recall (class 6): 0.6000
	 - Validation F1-score (class 6): 0.7500


Train loss: 0.0923705595672469

Epoch: 100%|██████████| 10/10 [11:17<00:00, 67.71s/it]


Epoch:  10
	 - Validation Accuracy: 0.7944
	 - Validation Accuracy (weighted): 0.7343

	 - Validation Precision (class 0): 0.6296
	 - Validation Recall (class 0): 0.6800
	 - Validation F1-score (class 0): 0.6538

	 - Validation Precision (class 1): 0.8500
	 - Validation Recall (class 1): 0.8095
	 - Validation F1-score (class 1): 0.8293

	 - Validation Precision (class 2): 0.7500
	 - Validation Recall (class 2): 0.9000
	 - Validation F1-score (class 2): 0.8182

	 - Validation Precision (class 3): 0.8636
	 - Validation Recall (class 3): 1.0000
	 - Validation F1-score (class 3): 0.9268

	 - Validation Precision (class 4): 1.0000
	 - Validation Recall (class 4): 0.8182
	 - Validation F1-score (class 4): 0.9000

	 - Validation Precision (class 5): 1.0000
	 - Validation Recall (class 5): 0.1667
	 - Validation F1-score (class 5): 0.2857

	 - Validation Precision (class 6): 1.0000
	 - Validation Recall (class 6): 0.8000
	 - Validation F1-score (class 6): 0.8889


Best results:

Epoch:  8
	 - 